### OneR algorithm

In [202]:
import pandas as pd
import numpy as np
import random

In [203]:
def generate_income():
    sums = [30, 40, 50]
    a = random.choice(sums)
    
    remaining_sums = [x for x in sums if x > a]
    remaining_sums.append(60)
    b = random.choice(remaining_sums)
    
    range = str(a) + 'K - ' + str(b) + 'K'
    return range


data = {
    'age': np.random.randint(20, 60, size=10),
    'gender': np.random.choice(['Male', 'Female'], size=10),
    'income_range': [generate_income() for _ in range(10)],
    'credit_card_insurance': np.random.choice([0, 1], size=10),
    'predicted_variable' : np.random.choice([0, 1], size=10)
}

# Create a DataFrame
#df = pd.DataFrame(data)

# Display the dataset
#print(df)


In [204]:
def get_nunique(df, predicted_variable):
    nunique = []
    for column in df.columns:
        if column != predicted_variable:
            nunique.append((column, df[column].nunique()))
    
    return nunique

def binary_split(unique_values, column, df, predicted_variable):
    to_be_sorted = []
    for value in unique_values:
        count = 0
        total = 0
        for index, row in df.iterrows():
            if row[column] == value:
                total += 1
                if row[predicted_variable] == 1:
                    count += 1
        
        accuracy = round(count / total *100)
        if accuracy > 50:
            prediction = 1
        else:
            prediction = 0
        to_be_sorted.append((column, value, prediction))
        to_be_sorted.sort(key=lambda x: x[1])
    
    best_accuracy = 0
    best_range = []
    for i in range(len(to_be_sorted)-1):
        total_1 = 0
        count_1 = 0
        range_1 = to_be_sorted[:i+1]
        range_2 = to_be_sorted[i+1:]

        for item in range_1:
            total_1 += 1
            if item[2] == 1:
                count_1 += 1

        total_2 = 0
        count_2 = 0
        for item in range_2:
            total_2 += 1
            if item[2] == 0:
                count_2 += 1

        total = (count_1 + count_2) / len(to_be_sorted)
        
        if total > best_accuracy:
            best_accuracy = total
            best_range = len(range_1)    
    
    split = to_be_sorted[best_range][1]
    return split, best_accuracy

def oneR(df, predicted_variable):
    nunique = get_nunique(df, predicted_variable)
    overall_accuracies = []
    overall_decision = []
    first_condition = []
    second_condition = []


    for item in nunique:
        column = item[0]
        unique_values = df[column].unique()
        overall_total = 0
        accuracies = []

        
        if item[1] < 7 or not isinstance(item[1], (int, float)):
            first_condition.append(item)
            for value in unique_values:
                yes = []
                no = []
                cant_decide = []
                count = 0
                total = 0
                overall_total = 0

                for index, row in df.iterrows():
                    overall_total += 1
                    if row[column] == value:
                        total += 1
                        if row[predicted_variable] == 1:
                            count += 1
                
                accuracy = round(count / total *100, 2)     
                accuracies.append((column, value, accuracy))

            for each in accuracies:
                if each[2] <= 50:
                    no.append(each)
                elif each[2] >= 100:
                    yes.append(each)
                else :
                    cant_decide.append(each)
            
            if len(yes) > len(no):
                no.extend(cant_decide)
            else:
                yes.extend(cant_decide)

            yes_updated = []
            for x, y, _ in yes:
                yes_updated.append((x, y, 1))

            no_updated = []
            for x, y, _ in no:
                no_updated.append((x, y, 0))
            decision = yes_updated + no_updated
            overall_decision.extend(decision)
            
            count = 0
            for column, value, prediction in decision:
                for index, row in df.iterrows():
                        if row[column] == value:
                            if row[predicted_variable] == prediction:
                                count += 1
            overall_accuracies.append((column, round(count / overall_total, 2)))

        else :
            second_condition.append(item)
            print("item 2. podmienka: ", item)
            all_values = df[column].tolist()
            split_number, accuracy = binary_split(all_values, column, df, predicted_variable)
            overall_accuracies.append((column, accuracy))
            overall_decision.append((column, split_number, 1))
        
    print(overall_accuracies)
    print(overall_decision)
    overall_accuracies.sort(key=lambda x: x[1], reverse=True)
    model = overall_accuracies[0]
    print("model based on:", model[0])

    good = []
    for each in overall_decision:
        if each[0] == model[0]:
            if each[2] == 1:
                print("1 is predicted for", each[1])
                good.append(each[1])
            
    predictions = []
    if model[0] in first_condition:
        for index, row in df.iterrows():
            if row[model[0]] in good:
                predictions.append(1)
            else:
                predictions.append(0)
    else:
        for index, row in df.iterrows():
            if row[model[0]] <= good[0]:
                predictions.append(1)
            else:
                predictions.append(0)
    
    print("predictions:", predictions)
    predicted = df[predicted_variable].tolist()
    print("actual:", predicted)

    differences = sum(pred != true_val for pred, true_val in zip(predictions, predicted))
    accuracy = round((len(predictions) - differences) / len(predictions) * 100, 2)
    print("accuracy:", accuracy, "%")

    return model[0], predictions, accuracy

In [205]:
X_test = pd.read_csv("X_test.csv", sep = ',')
y_test = pd.read_csv("y_test.csv", sep = ',')

merged_test_data = pd.concat([X_test, y_test], axis=1)

column_dtypes = merged_test_data.dtypes
print(column_dtypes)

data = merged_test_data.head(100)

oneR(data, predicted_variable='ack')

col_0                               int64
col_1                               int64
col_2                               int64
col_3                               int64
col_4                               int64
sex_F                             float64
sex_M                             float64
new_user_id                         int64
race_0                              int64
race_1                              int64
race_2                              int64
user_id                             int64
pct_mouse_click                   float64
pct_scroll_move                   float64
pct_doubleclick                   float64
pct_input                         float64
page_activity_duration            float64
pct_click                         float64
pct_mouse_move                    float64
pct_scrandom                      float64
pct_scroll_move_duration          float64
mouse_move_total_rel_distance     float64
product_ean                         int64
pct_rage_click                    

('pct_mouse_move',
 [0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1],
 67.0)

In [206]:
import copy

more_predictions = []


data_copy = copy.deepcopy(data)

more_predictions = []

for _ in range(5):
    model, predictions, accuracy = oneR(data_copy, predicted_variable='ack')
    if accuracy > 50:
        more_predictions.append(predictions)
    data_copy.drop(model, axis=1, inplace=True)

final_values = []

for col in zip(*more_predictions):
    count_0 = col.count(0)
    count_1 = col.count(1)
    
    final_value = 0 if count_0 > count_1 else 1
    final_values.append(final_value)

for each in more_predictions:
    print(each)
    
print(final_values)

ack = data['ack'].tolist()
correct = sum(1 for pred, true_val in zip(final_values, ack) if pred == true_val)
accuracy = (correct / len(ack)) * 100

print(f"Accuracy based on 'ack' column: {accuracy:.2f}%")


item 2. podmienka:  ('new_user_id', 30)
item 2. podmienka:  ('user_id', 28)
item 2. podmienka:  ('pct_mouse_click', 91)
item 2. podmienka:  ('pct_scroll_move', 91)
item 2. podmienka:  ('pct_doubleclick', 91)
item 2. podmienka:  ('pct_input', 90)
item 2. podmienka:  ('page_activity_duration', 91)
item 2. podmienka:  ('pct_click', 91)
item 2. podmienka:  ('pct_mouse_move', 91)
item 2. podmienka:  ('pct_scrandom', 91)
item 2. podmienka:  ('pct_scroll_move_duration', 91)
item 2. podmienka:  ('mouse_move_total_rel_distance', 91)
item 2. podmienka:  ('product_ean', 91)
item 2. podmienka:  ('pct_rage_click', 91)
item 2. podmienka:  ('pct_wild_mouse', 91)
item 2. podmienka:  ('session_duration', 91)
item 2. podmienka:  ('wild_mouse_duration', 91)
item 2. podmienka:  ('session_id', 91)
item 2. podmienka:  ('pct_click_product_info', 91)
item 2. podmienka:  ('scroll_move_total_rel_distance', 91)
item 2. podmienka:  ('total_load_time', 91)
item 2. podmienka:  ('screen_width', 7)
item 2. podmienka: